In [1]:
%matplotlib inline
import pandas as pd
import itertools
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class Dice(object):
    def __init__(self):
        self.dice = [
            [3,2,0,'g'],
            [2,2,0,'g'],
            [1,2,0,'g'],
            [1,0,1,'g'],
            [1,1,1,'g'],
            [2,1,1,'g'],
            [3,2,0,'b'],
            [2,0,1,'b'],
            [4,2,0,'b'],
            [3,1,1,'b'],
            [5,1,0,'b'],
            [2,1,0,'b'],
            [0,1,0,'r'],
            [0,2,0,'r'],
            [0,2,1,'r'],
            [0,2,0,'r'],
            [0,3,0,'r'],
            [0,3,0,'r'],
            [0,0,1,'y'],
            [1,2,0,'y'],
            [2,0,1,'y'],
            [1,1,1,'y'],
            [0,1,2,'y'],
            [2,1,0,'y'],
            [0,-1,0,'B'],
            [0,-1,0,'B'],
            [0,-2,0,'B'],
            [0,-2,0,'B'],
            [0,-3,0,'B'],
            [0,0,-1,'B'],
            [0,0,0,'W'],
            [0,-1,0,'W'],
            [0,0,-1,'W'],
            [0,-1,-1,'W'],
            [0,-1,-1,'W'],
            [0,'x','x','W']
        ]
        self.columns = ['rng', 'dmg', 'srg', 'col']
        self.base_df = pd.DataFrame(data=self.dice, columns=self.columns, dtype=int)
        self.colour_set = set(self.base_df['col'])
    
    def dodge_helper(self, combs):
        helper = list(zip(*combs))
        out = []
        for attr in helper:
            no_dodge = [type(z)==int for z in attr] # Checks that no dodges thrown
            if all(no_dodge):
                out += [max(0,sum(attr))]
            else:
                out += [0]
        return out
    
    
    def calc_options(self, die, given=()):
        assert all(dice in self.colour_set for dice in die), 'Check the dice you are searching for'
        out = []
        df = self.base_df
        for dice in die:
            part = df[df.col == dice]
            dice = []
            for n, r in part.iterrows():
                dice += [[r.rng, r.dmg, r.srg]]
            out += [dice]
        dice_combos = itertools.product(*out)
        dice_totals = []
        for combs in dice_combos:
            dice_totals += [self.dodge_helper(combs)]
        #assert len(dice_totals) == 6 ** len(die)
        df= pd.DataFrame(data=dice_totals, columns=self.columns[:-1])
        if given:
            attr, param = given
            return df[pd.to_numeric(df[attr], errors='coerce') >= param]
        return df

    
    def roll(self, die, summary=True):
        if summary:
            opts = self.calc_options(die)
            return opts.sample()
        out = []
        for dice in die:
            opt = self.base_df[self.base_df.col == dice].sample()
            out += [opt]
        out_df = pd.concat(out)
        total_row = out_df.sum()
        total_row.name = 'Total'
        return out_df.append(total_row)

    def opts_to_percent(self, opts, die):
        return round(opts / (6 **len(die)) * 100)
    
    def calc_prob(self, die, attr='dmg', points=1, given=()):
        opts = self.calc_options(die, given)
        opts = opts[attr].value_counts().sort_index()
        opts = opts.apply(lambda x: self.opts_to_percent(x, die))
        return opts
    
    def expected_values(self, die,rounding=1):
        opts = self.calc_options(die)
        average = opts.mean()
        deviation = opts.std()
        index = ['average', 'deviation']
        return np.round(pd.DataFrame(data=[average, deviation], index=index), rounding)
    
    def prep_for_graph(self, die, attr, given=()):
        opts = self.calc_options(die, given)
        try:
            grouped = opts.groupby(attr).size()
        except ValueError:
            grouped = pd.Series(name='dmg')        
        percentified = grouped.apply(lambda x: self.opts_to_percent(x, die))
        return percentified

    def graph(self, die, attr='dmg', kind='bar', given=()):
        if type(die) != list:
            die = [die]
        out = []
        for dice in die:
            out += [self.prep_for_graph(dice, attr, given)]
        out = pd.DataFrame(data=out,index=die).transpose()
        return out.plot(kind=kind)
    
    def chance_of_x(self,die, attr='dmg', given=()):
        if type(die) != list:
            die = [die]
        out = []
        for dice in die:
            probabilities = self.calc_prob(dice, attr=attr, given=given)
            out += [probabilities[::-1].cumsum()]
        return pd.DataFrame(data=out,index=die).transpose().sort_index()
            
    
d = Dice()

In [12]:
out = []
for dist in range(8):
    out += [d.chance_of_x(['rb'], attr='dmg', given=('rng', dist))]

x = pd.concat(out, axis=1, keys=range(8))


x

,0,1,2,3,4,5,6,7
,rb,rb,rb,rb,rb,rb,rb,rb
1,100,100,100,NaN,NaN,NaN,NaN,NaN
2,97,97,97,67,34,17,NaN,NaN
3,80,80,80,61,31,14,NaN,NaN
4,44,44,44,39,20,6,NaN,NaN
5,11,11,11,11,6,NaN,NaN,NaN


In [10]:
xx = d.base_df
xx[xx.col == 'B'].dmg.mean()

-1.5